In [1]:
#Loading/Ingestion
from llama_index.core import SimpleDirectoryReader
from llama_index.core.ingestion import IngestionPipeline
from llama_index.core.node_parser import JSONNodeParser


documents = SimpleDirectoryReader("./data").load_data()

pipeline = IngestionPipeline(transformations=[JSONNodeParser()])

nodes = pipeline.run(documents=documents)
nodes

# Transformations#
# After the data is loaded, you then need to process and transform your data before putting it into a storage system. These transformations include chunking, extracting metadata, and embedding each chunk. This is necessary to make sure that the data can be retrieved, and used optimally by the LLM.

# Transformation input/outputs are Node objects (a Document is a subclass of a Node). Transformations can also be stacked and reordered.

[TextNode(id_='59de8771-cfc5-43c8-9fde-ca71a1c696bd', embedding=None, metadata={'file_path': '/home/shivangvora/Legal-AI-Assistant/data/art15.json', 'file_name': 'art15.json', 'file_type': 'application/json', 'file_size': 3533, 'creation_date': '2025-05-16', 'last_modified_date': '2025-05-16'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='61047104-657a-42fe-91f6-447a8b142869', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'file_path': '/home/shivangvora/Legal-AI-Assistant/data/art15.json', 'file_name': 'art15.json', 'file_type': 'application/json', 'file_size': 3533, 'creation_date': '2025-05-16', 'last_modified_date': '2025-05-16'}, hash='bc1bfaf991b31ab4f4d5ae12716657fedaad156c6c3a668fe499378232aa864

In [5]:
#Indexing/Embedding for Ollama
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings
from llama_index.core import VectorStoreIndex

Settings.llm = Ollama(model="gemma3:1b", request_timeout=600.0)
Settings.embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-MiniLM-L6-v2")

index = VectorStoreIndex(nodes)

In [3]:
#Storing
index.storage_context.persist(persist_dir="index-persistent")


from llama_index.core import StorageContext, load_index_from_storage

# rebuild storage context
storage_context = StorageContext.from_defaults(persist_dir="index-persistent")

# load index
index = load_index_from_storage(storage_context)

In [ ]:
#Querying
from llama_index.core import VectorStoreIndex, get_response_synthesizer
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.postprocessor import SimilarityPostprocessor
# configure retriever
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=10,
    similarity_metric="cosine",
)

# configure response synthesizer
response_synthesizer = get_response_synthesizer()

# assemble query engine
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
    node_postprocessors=[SimilarityPostprocessor(similarity_cutoff=0.7)],
)

query_engine = index.as_query_engine()


In [8]:
response = query_engine.query(
    "Ajay is a shop owner who does not allow Alex a customer to enter his store because Alex is from a different religion. Is this allowed under Article 15 of the Indian Constitution? Explain your reasoning."
)
print(response)

The provided text does not explicitly state whether Ajay is allowed to deny a customer entry to his store based on religion. However, the text outlines the State’s obligation to ensure equal treatment for all citizens, regardless of religion, and explicitly prohibits discrimination based on religion. This suggests the State has the power to intervene to prevent discriminatory practices, including those related to religious affiliation. Therefore, while the text doesn’t directly address the situation, it establishes a legal framework that could be interpreted as allowing such a restriction.
